# Machine Learning: Exercise session 04

In this exercise session we will focus on Ridge and Lasso regression. You will learn how to fit, predict, and cross-validate these models.

In the first problem, we will continue using the housing dataset where we added several interaction terms, and one additional variable named `X1`. You can download the data from Moodle in this week's section.

The second problem is theoretical and not due for hand-in. There, we will derive the closed-form solution for the Ridge regression (notice that Lasso regression has no closed-form solution).

## Problem 1

### 0. Load data and create a test set

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

* Import the clean housing dataset with `pd.read_csv` and take a quick look at it, to verify that it is in the desired shape (display the dataframe, check its "infos").

In [3]:
df=pd.read_csv("housing_interactions.csv")

FileNotFoundError: ignored

In [ ]:
df
df.info()

* Separate the dataframe into the features `X` and the target variable `y`: (remember, we want to predict the median house value, given the other variables)

In [ ]:
X = df[["longitude","latitude", "housing_median_age","median_income","rooms_per_bedroom","rooms_per_household","people_per_household","ocean_proximity_enc"]]
Y = df[["median_house_value"]]

* Using the function `train_test_split`, split the dataset into training and test set. Set `test_size = 0.95` and `random_state = 12`. 

*Notice that we choose a very large fraction of test data because we want to see whether the Ridge and Lasso regression can handle the high-dimensional setting.*

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(X,Y, test_size=.95, random_state=12)

### 1. Define models

* Using the class `Ridge` from `sklearn.linear_model`, instantiate a ridge regression object with penalty parameter equal to 0.1. Furthermore, set `fit_intercept=True`, and `normalize=True`. Call this object `ridge_reg`.

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.1, fit_intercept=True, normalize= True)


* Using the class `Lasso` from `sklearn.linear_model`, instantiate a ridge regression object with penalty parameter equal to 0.1. Furthermore, set `fit_intercept=True`, `normalize=True`, and `max_iter=1e5`. Call this object `lasso_reg`.

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg=Lasso(alpha=0.1, fit_intercept= True, normalize = True, max_iter=1e5)

What is the role of the option `fit_intercept`?
*The idea is too ask lasso to add the beta0 (the intercept) to its penality term*.

### 2. Fit Ridge regression

* Fit the ridge regression on the training data previously created.

In [ ]:
ridge_reg.fit(x_train, y_train)

* Compute the root mean square error of the fitted model on both the training and test set. What do you observe?
Plus grand erreur de prédiction sur le testing data set que le training. Mais pas égal à 0 pour le training, car ridge empêche overfitting.

In [ ]:
from sklearn.metrics import mean_squared_error
mse_train_ridge=mean_squared_error(y_train, ridge_reg.predict(x_train), squared=False)
mse_train_ridge

In [ ]:
mse_test_ridge=mean_squared_error(y_test, ridge_reg.predict(x_test), squared=False)
mse_test_ridge

In [ ]:
# R^2: 1 - E[(Y - Y_hat)^2]/E[(Y - E[Y])^2] 
# (proportion of variability of Y explained by fitted model)
1 - mean_squared_error(y_test, ridge_reg.predict(x_test)) / np.var(y_test)

### 4. Plot Ridge regression coefficients

We now want to plot the coefficients of the ridge regression for different values of the penalty parameter.

* Create a grid, named `grid`, of penalty values ranging from `1e-4` to `1e4`.

In [ ]:
ls

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
# grid = np.arange(1e-4, 1e4, 100)
grid=np.array([1e-4*10**k for k in range(9)]) #why like this ?


* Fill in the `??` below.

In [ ]:
coefs_ridge = []
for alph in grid:
    ridge_reg.set_params(alpha= alph)
    ridge_reg.fit(x_train, y_train) #train ou test? train ! Choisir l'hyperparameter fait partie du training.
    coefs_ridge.append(ridge_reg.coef_[0])

In [ ]:
plt.figure(figsize=(17,9))
plt.plot(np.log10(grid), coefs_ridge)
plt.legend(X.columns)

### 5. Fit Lasso regression

* Fit the lasso regression on the training data previously created.

In [ ]:
lasso_reg.fit(x_train, y_train)

* Compute the root mean square error of the fitted model on both the training and test set. What do you observe?

In [ ]:
mse_train_lasso=mean_squared_error(y_train, lasso_reg.predict(x_train), squared=False)
mse_train_lasso

In [ ]:
mse_train_lasso/mse_train_ridge*100

*mse on train with lasso is $96\%$ of the ridge mse. So they are fairly similar*

In [ ]:
mse_test_lasso=mean_squared_error(y_test, lasso_reg.predict(x_test), squared=False)
mse_test_lasso

In [ ]:
mse_test_lasso/mse_test_ridge*100

*mse on test with lasso is $104\%$ of the ridge mse. So they are fairly similar*

### 6. Plot Lasso coefficients

As above, we want to plot the coefficients of the lasso regression for different values of the penalty parameter.

* Fill in the `??` below.

In [ ]:
coefs_lasso = []
for alph in grid:
    lasso_reg.set_params(alpha=alph)
    lasso_reg.fit(x_train, y_train) #train or test?
    coefs_lasso.append(lasso_reg.coef_)

In [ ]:
plt.figure(figsize=(17,9))
plt.plot(np.log10(grid), coefs_lasso)
plt.legend(X.columns)

### 7. Prepare CV

At this point, we want to find the optimal penalty parameter for our models.
We will use 10-fold cross-validation.

* Using the `KFold` class from `sklearn.model_selection` and instatiate an object named `folds`. Set the number of splits equal to 10, the random seed equal to 42, and make sure to shuffle the rows.

In [ ]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, shuffle=True, random_state=42)


### 8. CV for Ridge

* Define a grid of penalty parameters for the ridge regression. You can use the same values used for the grid above. Name the object `ridge_grid`.

In [ ]:
ridge_grid=grid

* Create a dictionary with the key-value pair `"alpha"`, `ridge_grid`. What is the use of this object?

In [ ]:
hyper_parameters={"alpha" : ridge_grid}

*It will be used to test different alpha values of the ridge fit.*

* Import the class `GridSearchCV` from `sklearn.model_selection` and use it to instantiate a **cross-validation object** for the ridge regression. Make sure to include the following parameters: `estimator`, `param_grid`, `scoring`, `cv`.

In [ ]:
from sklearn.model_selection import GridSearchCV
ridge_cv = GridSearchCV(estimator=ridge_reg, param_grid=hyper_parameters, scoring="neg_mean_squared_error",
                       cv=folds)

* Run the cross-validation by calling the `fit` method of the **cross-validation object** that you created at the point before.

In [ ]:
ridge_cv.fit(x_train,y_train)

* We want to plot the cross-validation error against different values of the penalty parameter. Fill in the `??`.

In [ ]:
mean_scores = -ridge_cv.cv_results_["mean_test_score"]
std_scores = ridge_cv.cv_results_["std_test_score"] /np.sqrt(10)
alphas = np.array([k for k in ridge_cv.cv_results_["param_alpha"].data]) 

best_index = np.argmin(mean_scores) # index of the alpha with the lowest MSE estimate
min_alpha_ridge = ridge_grid[best_index]
print("Minimum alpha:", min_alpha_ridge)



In [ ]:
ridge_cv.cv_results_

In [ ]:
plt.errorbar(x = np.log10(alphas), y = mean_scores, yerr = std_scores, fmt='o', capsize=3)

plt.axvline(np.log10(min_alpha_ridge), ls='dotted', color="grey")#vertical line at the k yielding minimum CV MSE

plt.title("Ridge regressor CV error")
plt.xlabel('log(lambda)')
plt.ylabel('Mean Squared Error')
plt.show()

### 9. CV for Lasso

* Define a grid of penalty parameters for the lasso regression. You can use the same values used for the grid above. Name the object `lasso_grid`.

In [ ]:
lasso_grid=ridge_grid

* Create a dictionary with the key-value pair `"alpha"`, `lasso_grid`. What is the use of this object?

In [ ]:
hyper_parameters_lasso = {"alpha": lasso_grid}

* Import the class `GridSearchCV` from `sklearn.model_selection` and use it to create a **cross-validation object** for the lasso regression. Make sure to include the following parameters: `estimator`, `param_grid`, `scoring`, `cv`.

In [ ]:
from sklearn.model_selection import GridSearchCV
lasso_cv=GridSearchCV(estimator=lasso_reg, param_grid=hyper_parameters_lasso, scoring = "neg_mean_squared_error", cv = folds)


* Run the cross-validation by calling the `fit` method of the **cross-validation object** that you created at the point before.

In [ ]:
lasso_cv.fit(x_train,y_train)

* We want to plot the cross-validation error against different values of the penalty parameter. Fill in the `??`.

In [ ]:
# Choose best model according to 1-se rule
mean_scores = -lasso_cv.cv_results_["mean_test_score"]
std_scores = lasso_cv.cv_results_["std_test_score"] /np.sqrt(10)
alphas = lasso_cv.cv_results_["param_alpha"].data
alphas = pd.to_numeric(alphas)

best_index = np.argmin(mean_scores)
min_alpha_lasso = alphas[best_index]
print("Minimum alpha:", min_alpha_lasso)

In [ ]:
plt.errorbar(x=np.log10(alphas),  y=-ridge_cv.cv_results_["mean_test_score"], yerr = ridge_cv.cv_results_["std_test_score"], fmt='o', capsize=3)

plt.axvline(np.log10(min_alpha_lasso), ls='dotted', color="grey")#vertical line at the k yielding minimum CV MSE

plt.title("Lasso regressor CV error")
plt.xlabel('log(lambda)')
plt.ylabel('Mean Squared Error')
plt.show()

### 10. Compute performance of the two best models

* Given the optimal tuning parameter of the ridge and lasso regression, refit both models on the entire training data set. Furthermore, compute their root mean square error on the test set. How do they compare to the errors obtained before performing the cross-validation?

_Hint_: As optimal tuning parameter, choose the one that minimizes the Mean Squared Error and not the one according to the 1-standard error rule.

In [ ]:
# Refit ridge on training and evaluate on test
best_ridge_reg=Ridge(alpha=min_alpha_ridge, fit_intercept= True, normalize = True, max_iter=1e5)
best_ridge_reg.fit(x_train,y_train)

final_mse_test_ridge=mean_squared_error(y_test, best_ridge_reg.predict(x_test), squared=False)
print("the score of the ridge regression with the best hyperparameter is:", final_mse_test_ridge, "while when not choosing the best hyperparameter, its score was",mse_test_ridge)

In [ ]:
# Refit lasso on training and evaluate on test
best_lasso_reg=Lasso(alpha=min_alpha_lasso, fit_intercept= True, normalize = True, max_iter=1e5)
best_lasso_reg.fit(x_train,y_train)

final_mse_test_lasso=mean_squared_error(y_test, best_lasso_reg.predict(x_test), squared=False)
print("the score of the lasso regression with the best hyperparameter is:", final_mse_test_lasso, "while when not choosing the best hyperparameter, its score was",mse_test_lasso)

## Problem 2

In this problem, you are asked to derive the closed-form solution for the ridge regression coefficients.

Let $\mathbf{X}\in\mathbb{R}^{n\times p}$ denote the matrix of predictors, and let $\mathbf{y}\in\mathbb{R}^n$ denote the target vector.

The optimal ridge regression coefficient vector $\beta^*$, with parameter $\lambda > 0$, is defined as

$$\beta^* := \arg\min_{\beta\in\mathbb{R}^p} \ (\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) + \lambda \beta ^T\beta.$$

Show that 


$$\beta^*  = (\mathbf{X}^T\mathbf{X} + \lambda \mathbf{I}_p)^{-1} \mathbf{X}^T \mathbf{y},$$

where $\mathbf{I}_p$ denote the identity matrix of size $p\times p$.

$\newcommand{\mby}{\mathbf{y}}$
We want to find the $β^*$ such that $(\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) + \lambda \beta ^T\beta.$ is minimized. For this, we first compute $\frac{\partial}{\partial β}\left((\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) + \lambda \beta ^T\beta\right)$:

$\begin{align*}
\frac{\partial}{\partial β}\left((\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) + \lambda \beta ^T\beta\right) &= \frac{\partial}{\partial β}\left(\mathbf{y}^T\mathbf{y}-\mathbf{y}^T\mathbf{X}β-β^T\mathbf{X}^T\mathbf{y}+β^T\mathbf{X}^T\mathbf{X}β+λβ^Tβ\right)\\
&=-\mathbf{y}^T\mathbf{X}-\underbrace{\mathbf{y}^T\mathbf{X}}_{\frac{\partial}{\partial β}(β^tA)=A^T}+\underbrace{2\mathbf{X}^T\mathbf{X}β}_{\frac{\partial}{\partial β}((Aβ)^TAβ)=2Aβ} +2λβ\\
&=-2\mathbf{y}^T\mathbf{X}+2\mathbf{X}^T\mathbf{X}β +2λ
\end{align*}$
Now we equate it to $0$:
$\begin{align*}
-2\mathbf{y}^T\mathbf{X}+2\mathbf{X}^T\mathbf{X}β +2λβ&=0\\
\Leftrightarrow-\mathbf{y}^T\mathbf{X}+\mathbf{X}^T\mathbf{X}β +λβ&=0\\
\Leftrightarrow \left(\mathbf{X}^T\mathbf{X}+λId_{p}\right)β=\mathbf{y}^T\mathbf{X}\\
\Leftrightarrow β=\left(\mathbf{X}^T\mathbf{X}+λId_{p}\right)^{-1}\mathbf{y}^T\mathbf{X}
\end{align*}$

We then check that $\frac{\partial}{\partial β^T}\left(-2\mathbf{y}^T\mathbf{X}+2\mathbf{X}^T\mathbf{X}β +2λβ\right)$ is positive definite:
$$\frac{\partial}{\partial β^T}\left(-2\mathbf{y}^T\mathbf{X}+2\mathbf{X}^T\mathbf{X}β +2λβ\right)=2\mathbf{X}\mathbf{X}^T +2λId_{p}$$

And so
$$\mathbf{v}^T\left(2\mathbf{X}\mathbf{X}^T +2λId_{p}\right)\mathbf{v}=2\underbrace{\mathbf{v}^T\mathbf{X}\mathbf{X}^T\mathbf{v}}_{=||\mathbf{X}^Tv||^2>0}+2\underbrace{λ}_{>0}\overbrace{v^Tv}^{=||v||^2>0}>0$$